In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS PARA TU TAREA ---
GENRES = ['rock']
YEARS = [2010, 2024] 

In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')

In [4]:
# Años para iterar
YEARS = [2010, 2024] 

# Términos optimizados para una búsqueda de ALTO VOLUMEN

query_terms = [
    'Rock',
    'Rock Alternativo',
    'indie',
    'Pop Rock'
       # Entre comillas para buscar la frase exacta (útil para el 2010)
]

# 4. Inicializa el cliente de Spotify
try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None

✅ Conexión a Spotify exitosa.


In [5]:
# NUEVA CELDA: Comprobación de autenticación
print(type(sp)) 

<class 'spotipy.client.Spotify'>


In [6]:
# Prueba 2 Extraccion Spoti
# --- DEFINICIÓN DE LA FUNCIÓN CON DOBLE BUCLE (MÁXIMA ROBUSTEZ Y LÍMITES) ---

def extract_spotify_data_indie_ampliada(sp, years, query_terms): 
    """
    VERSION FINAL LIMITADA: Itera primero por AÑO y luego por CADA TÉRMINO de género.
    Máximo total: 500 canciones únicas. Máximo por año: 250 canciones únicas.
    """
    
    spotify_data = []
    MAX_TOTAL_SONGS = 500 # Nuevo límite global
    MAX_SONGS_PER_YEAR = 250 # Nuevo límite por año
    
    for year in years:
        
        # 1. VERIFICACIÓN DEL LÍMITE GLOBAL ANTES DE EMPEZAR EL AÑO
        if len(spotify_data) >= MAX_TOTAL_SONGS:
            print(f"🛑 LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Saltando el año {year}.")
            break
            
        year_range = f'{year}-{year}'
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")
        
        songs_added_in_year = 0 # Contador para este año
        
        # BUENO 1: Iteramos sobre cada término de búsqueda 
        for term in query_terms:
            
            # 2. VERIFICACIÓN DEL LÍMITE ANUAL
            if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                print(f"🛑 LÍMITE ANUAL ALCANZADO ({MAX_SONGS_PER_YEAR}). Terminando la búsqueda para el año {year}.")
                break
            
            query = f'{term} year:{year_range}'
            print(f"   -> Buscando con término: {term}")
            
            try:
                # La API siempre trae un máximo de 50 por llamada
                results = sp.search(q=query, type='track', limit=50) 
                
                total_resultados_api = results['tracks']['total'] if results and 'tracks' in results else 0
                print(f"   -> Spotify reporta {total_resultados_api} resultados para '{term}'.")
                
                songs_added_in_term = 0
                
                # BUENO 2: Bucle de Paginación
                while results: # Mientras haya más resultados (no ponemos un límite aquí para exprimir los 1000)
                    
                    for track in results['tracks']['items']:
                        
                        track_id = track['id']
                        
                        # 3. VERIFICACIÓN DE LÍMITES DENTRO DEL BUCLE INTERNO
                        
                        # A. Límite Global alcanzado
                        if len(spotify_data) >= MAX_TOTAL_SONGS:
                            print(f"🛑 LÍMITE GLOBAL ({MAX_TOTAL_SONGS}) alcanzado. Terminando.")
                            return pd.DataFrame(spotify_data) # Terminamos inmediatamente
                            
                        # B. Límite Anual alcanzado
                        if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                            break # Salta al siguiente término de búsqueda
                        
                        # C. Deduplicación
                        # Usamos una forma más eficiente de comprobar duplicados si el volumen es grande
                        if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                            continue
                            
                        # 4. Añadir la canción
                        data_row = {
                            'Artista': track['artists'][0]['name'],
                            'Género musical': f'indie (Origen: {term})',
                            'Tipo': 'Canción',
                            'Nombre': track['name'],
                            'Año de lanzamiento': year,
                            'ID_Spotify': track_id,
                            'ID_Album': track['album']['id']
                        }
                        spotify_data.append(data_row)
                        songs_added_in_term += 1
                        songs_added_in_year += 1 # Contar para el límite anual

                    # Paginación
                    if results['tracks']['next']:
                        results = sp.next(results['tracks'])
                    else:
                        results = None
                    
                    # Aumentamos la pausa para evitar el Rate Limiting
                    time.sleep(1.5) 
                
                print(f"      -> Añadidas {songs_added_in_term} canciones ÚNICAS de '{term}'.")
                
            except Exception as e:
                print(f"   -> Error al buscar con término '{term}': {e}")
                time.sleep(3) # Pausa más larga después de un error para recuperarse
            
    return pd.DataFrame(spotify_data)

In [7]:
# --- LLAMADA AL CÓDIGO (CELDA 3 CORREGIDA) ---

if sp:
    # Eliminamos el argumento 'max_total_songs=500'
    df_spotify_raw = extract_spotify_data_indie_ampliada(sp, YEARS, query_terms) 
    
    print("\n--- RESULTADO FINAL DE indie (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}") 
    print(df_spotify_raw.head(10))


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2010 ---
   -> Buscando con término: Rock
   -> Spotify reporta 42 resultados para 'Rock'.
      -> Añadidas 42 canciones ÚNICAS de 'Rock'.
   -> Buscando con término: Rock Alternativo
   -> Spotify reporta 39 resultados para 'Rock Alternativo'.
      -> Añadidas 39 canciones ÚNICAS de 'Rock Alternativo'.
   -> Buscando con término: indie
   -> Spotify reporta 100 resultados para 'indie'.
      -> Añadidas 94 canciones ÚNICAS de 'indie'.
   -> Buscando con término: Pop Rock
   -> Spotify reporta 100 resultados para 'Pop Rock'.
      -> Añadidas 75 canciones ÚNICAS de 'Pop Rock'.

--- INICIANDO EXTRACCIÓN PARA EL AÑO 2024 ---
   -> Buscando con término: Rock
   -> Spotify reporta 25 resultados para 'Rock'.
      -> Añadidas 25 canciones ÚNICAS de 'Rock'.
   -> Buscando con término: Rock Alternativo
   -> Spotify reporta 100 resultados para 'Rock Alternativo'.
      -> Añadidas 100 canciones ÚNICAS de 'Rock Alternativo'.
   -> Buscando con término: i

In [8]:
# VERIFICACIÓN DE CANCIONES
print(f"Número de canciones encontradas en df_spotify_raw: {len(df_spotify_raw)}")
print(f"Primeras filas (head) de df_spotify_raw:\n{df_spotify_raw.head()}")

Número de canciones encontradas en df_spotify_raw: 500
Primeras filas (head) de df_spotify_raw:
             Artista        Género musical     Tipo  \
0      George McCrae  indie (Origen: Rock)  Canción   
1              Falco  indie (Origen: Rock)  Canción   
2           Skrillex  indie (Origen: Rock)  Canción   
3  Far East Movement  indie (Origen: Rock)  Canción   
4           Deftones  indie (Origen: Rock)  Canción   

                                          Nombre  Año de lanzamiento  \
0                                 Rock Your Baby                2010   
1                                Rock Me Amadeus                2010   
2  Rock 'n' Roll (Will Take You to the Mountain)                2010   
3                                      Rocketeer                2010   
4                                  Rocket Skates                2010   

               ID_Spotify                ID_Album  
0  6pMHMzwTsWIxLigr54GGVm  1Vmgke5bm1op8mUnbvd6wY  
1  0wbdaBHi8t5e7EqXV88nka  4nZCwWIMM

In [9]:
# --- CELDA COMPLETA: EXTRACCIÓN Y UNIFICACIÓN DE ÁLBUMES ---

# Asegúrate de que las librerías necesarias (pandas, time) y la variable 'sp' estén disponibles
import pandas as pd
import time
# Nota: La variable 'sp' debe estar definida por la Celda 1.

# -------------------------------------------------------------------
# BLOQUE 1: DEFINICIÓN DE LA FUNCIÓN
# -------------------------------------------------------------------

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # El set ya se crea cuando se llama a la función con album_ids_list.
    unique_album_ids = album_ids
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] 
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa para no saturar la API
            
        except Exception as e:
            # Capturamos errores como álbumes que han sido eliminados de Spotify
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)

# -------------------------------------------------------------------
# BLOQUE 2: EJECUCIÓN Y FUSIÓN
# -------------------------------------------------------------------

# Verificación de que el DataFrame de canciones está disponible
if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    print("\n--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---")
    
    # 1. VERIFICACIÓN CRÍTICA: La columna 'ID_Album' debe existir.
    if 'ID_Album' not in df_spotify_raw.columns:
        print("❌ ERROR CRÍTICO: La columna 'ID_Album' NO se encontró. La Celda 3 falló al guardar el ID del álbum.")
    else:
        
        # 2. Obtener IDs únicos válidos de los álbumes
        album_ids_list = df_spotify_raw['ID_Album'].dropna().unique() 
        
        if len(album_ids_list) == 0:
            print("❌ LISTA VACÍA: Se encontraron canciones, pero 0 IDs de álbumes válidos. No se puede continuar.")
        else:
            print(f"✅ Se encontraron {len(album_ids_list)} IDs de álbumes únicos. Extrayendo ahora...")
            
            # 3. Ejecutar la función de extracción (Tardará unos minutos)
            df_album_details = extract_album_details(sp, album_ids_list)
            
            # 4. Fusionar (JOIN) los detalles del álbum con los datos de las canciones
            df_spotify_final = pd.merge(
                df_spotify_raw,
                df_album_details[['ID_Album', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album']],
                on='ID_Album',
                how='left' 
            )
            
            print(f"\n--- PROCESO DE ÁLBUMES COMPLETADO ---")
            print(f"DataFrame intermedio 'df_spotify_final' creado con {len(df_spotify_final)} filas.")
            print(df_spotify_final.head())
            
else:
    print("❌ ERROR: El DataFrame de canciones (df_spotify_raw) no está disponible. Vuelve a ejecutar la Celda 3.")


--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---
✅ Se encontraron 276 IDs de álbumes únicos. Extrayendo ahora...

--- INICIANDO EXTRACCIÓN DE METADATOS DE 276 ÁLBUMES ---
✅ Extracción de 276 álbumes completada.

--- PROCESO DE ÁLBUMES COMPLETADO ---
DataFrame intermedio 'df_spotify_final' creado con 500 filas.
             Artista        Género musical     Tipo  \
0      George McCrae  indie (Origen: Rock)  Canción   
1              Falco  indie (Origen: Rock)  Canción   
2           Skrillex  indie (Origen: Rock)  Canción   
3  Far East Movement  indie (Origen: Rock)  Canción   
4           Deftones  indie (Origen: Rock)  Canción   

                                          Nombre  Año de lanzamiento  \
0                                 Rock Your Baby                2010   
1                                Rock Me Amadeus                2010   
2  Rock 'n' Roll (Will Take You to the Mountain)                2010   
3                                      Rocketeer                2

In [27]:
def extract_lastfm_artist_details(artist_name):
    """Extrae biografía y estadísticas de popularidad de un artista."""
    
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'artist' in data:
            artist_info = data['artist']
            
            # Extraer los datos relevantes (Biografía, Popularidad/Plays)
            biography = artist_info.get('bio', {}).get('summary', 'N/A')
            playcount = artist_info.get('stats', {}).get('playcount', '0')
            listeners = artist_info.get('stats', {}).get('listeners', '0')
            
            return {
                'Artista': artist_name,
                'Biografia_Resumen': biography,
                'Playcount_LastFM': playcount,
                'Listeners_LastFM': listeners,
            }
    except Exception:
        # En caso de error de conexión o artista no encontrado, devuelve None
        pass 
        
    return None



In [28]:
# --- Ejecución para Artistas Únicos ---

if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    # Obtener la lista única de artistas para evitar búsquedas repetidas
    unique_artists = df_spotify_raw['Artista'].unique()
    lastfm_results = []
    
    print("\n--- EXTRACCIÓN DETALLES LAST.FM ---")
    
    for artist in unique_artists: 
        details = extract_lastfm_artist_details(artist)
        if details:
            lastfm_results.append(details)
        time.sleep(0.5) # Pausa crucial para no sobrecargar la API

    df_lastfm = pd.DataFrame(lastfm_results)
    
    # 5. UNIFICACIÓN DE DATOS
    # Fusionar los datos de Spotify con los detalles de Last.fm
    df_final_indie = pd.merge(
        df_spotify_raw, 
        df_lastfm[['Artista', 'Playcount_LastFM', 'Listeners_LastFM']],
        on='Artista', 
        how='left' # Usamos left join para mantener todas las canciones de Spotify
    )
    
    print("\n--- DATAFRAME FINAL DE INDIE LISTO PARA ALMACENAMIENTO ---")
    print(df_final_indie.head())
    print(f"Total de registros finales para Indie: {len(df_final_indie)}")


--- EXTRACCIÓN DETALLES LAST.FM ---

--- DATAFRAME FINAL DE INDIE LISTO PARA ALMACENAMIENTO ---
             Artista        Género musical     Tipo  \
0      George McCrae  indie (Origen: Rock)  Canción   
1              Falco  indie (Origen: Rock)  Canción   
2           Skrillex  indie (Origen: Rock)  Canción   
3  Far East Movement  indie (Origen: Rock)  Canción   
4           Deftones  indie (Origen: Rock)  Canción   

                                          Nombre  Año de lanzamiento  \
0                                 Rock Your Baby                2010   
1                                Rock Me Amadeus                2010   
2  Rock 'n' Roll (Will Take You to the Mountain)                2010   
3                                      Rocketeer                2010   
4                                  Rocket Skates                2010   

               ID_Spotify                ID_Album Playcount_LastFM  \
0  6pMHMzwTsWIxLigr54GGVm  1Vmgke5bm1op8mUnbvd6wY          1778396   

In [31]:
# --- BLOQUE DE FUSIÓN Y GUARDADO CORREGIDO ---

# 1. Aseguramos que tenemos df_spotify_final y df_lastfm en memoria.

if 'df_spotify_final' in locals() and 'df_lastfm' in locals():
    
    # Lista de columnas a fusionar (¡EXCLUYE Artistas_Similares!)
    lastfm_cols_to_merge = ['Artista', 'Biografia_Resumen', 'Playcount_LastFM', 'Listeners_LastFM']
    
    # 2. Re-ejecutar el merge (fusión)
    df_proyecto_final = pd.merge(
        df_spotify_final, 
        # Seleccionamos solo las columnas que sabemos que existen:
        df_lastfm[lastfm_cols_to_merge], # <-- ¡CRUCIAL!
        on='Artista', 
        how='left'
    )
    
    # 3. Guardar el archivo CSV (Tu Celda 5 original)
    output_filename = 'indie_data_2010_2024.csv'
    
    df_proyecto_final.to_csv(
        output_filename, 
        index=False,        
        encoding='utf-8'    
    )
    
    print(f"\n🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉")
# ...


🎉 ¡FASE 1 FINALIZADA CON ÉXITO! 🎉
